# Deep and Reinforcement Learning 2024/2025 (M.IA003), FEUP/FCUP

## Deep Learning Project 
## **Develop deep learning discriminative and generative models, applied to the context of “deep fakes”**

work done by:
- Michal Kowalski up
- Pedro Pereira up
- Pedro Azevedo up201905966

## 1.1) Import Necessary

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os

In [ ]:
# Define dataset directories
data_dir = "data"
real_dir = os.path.join(data_dir, "real")
fake_dir = os.path.join(data_dir, "fake")

## 1.2) Basic EDA

In [ ]:
from _eda import base_stats

base_stats(real_dir, fake_dir)

## 1.3) File Standardization 

In [1]:
from _preprocess import standardize_files

standardize_files('data/real', (128, 128))
standardize_files('data/fake', (128, 128))

Skipping 43963045_1995-04-27_2015.jpg: name 'output_dir' is not defined
Skipping 1698400_1958-12-10_2008.jpg: name 'output_dir' is not defined
Skipping 3940807_1899-06-14_1964.jpg: name 'output_dir' is not defined
Skipping 37628434_1991-10-02_2013.jpg: name 'output_dir' is not defined
Skipping 3949129_1974-11-13_2008.jpg: name 'output_dir' is not defined
Skipping 28138518_1991-07-04_2013.jpg: name 'output_dir' is not defined
Skipping 9069371_1986-03-19_2013.jpg: name 'output_dir' is not defined
Skipping 20315592_1987-03-15_2010.jpg: name 'output_dir' is not defined
Skipping 1320502_1981-03-29_2011.jpg: name 'output_dir' is not defined
Skipping 42429529_1961-12-18_2012.jpg: name 'output_dir' is not defined
Skipping 8115150_1976-07-29_2007.jpg: name 'output_dir' is not defined
Skipping 2213548_1935-03-03_1972.jpg: name 'output_dir' is not defined
Skipping 1257746_1927-04-10_2009.jpg: name 'output_dir' is not defined
Skipping 10181805_1956-06-05_1984.jpg: name 'output_dir' is not defined


KeyboardInterrupt: 

In [ ]:
real_dir = os.path.join(data_dir, "real_normalized_128")
fake_dir = os.path.join(data_dir, "fake_normalized_128")
base_stats(real_dir, fake_dir)


Real Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((512, 512), 30000)]

Fake Images Summary:
Total images: 30000
Formats: Counter({'JPEG': 30000})
Color Modes: Counter({'RGB': 30000})
Top 5 Sizes: [((512, 512), 30000)]


## 1.4) Baseline

In [1]:
from _model import build_gan, train_gan

d, g, gan = build_gan()

2025-03-02 16:42:08.629949: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 16:42:08.635649: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-02 16:42:08.650616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740933728.673823   12094 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740933728.681077   12094 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 16:42:08.704977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
import glob
# Get all image file paths
image_paths = glob.glob("./data/real_normalized/*.jpg")  # Change to *.png if needed

def load_and_preprocess(image_path):
    image = tf.io.read_file(image_path)  
    image = tf.image.decode_jpeg(image, channels=3)  # Change to decode_png if needed
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0  # Normalize to [-1, 1]
    return image


In [5]:
gan.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 512, 512, 3)    │     6,881,139 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     1,102,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,983,412 (30.45 MB)

 Trainable params: 6,749,971 (25.75 MB)

 Non-trainable params: 1,233,441 (4.71 MB)

In [8]:
BATCH_SIZE = 32
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [9]:
train_gan(dataset, d, g, gan, epochs=100, batch_size=BATCH_SIZE)

: 